In [13]:
import json
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import roc_curve, auc

In [14]:
file = "/Users/shutinggu/Desktop/1001-project/renttherunway_final_data.json"
df = pd.read_json(file,lines=True)


In [15]:
# transfer bust size into bust size and cups
df=df.dropna()
df['cups'] = df['bust size'].str.extract(r'([a-z])')
labels = df['cups'].astype('category').cat.categories.tolist()
replace_map = {'cups' : {k: v for k,v in zip(labels,list(range(1,len(labels)+1)))}}
print('--- map cups to numerical value ---')
print(replace_map)
df.replace(replace_map, inplace=True)
df['bust size'] = df['bust size'].str.extract('(\d+)').astype(int)
# remove lbs after 'weight'
df['weight'] = df['weight'].str.extract('(\d+)').astype(int)
# parse height to usable numerical format
def parse_height(ht):
    ht_ = ht.split("' ")
    ft_ = float(ht_[0])
    in_ = float(ht_[1].replace("\"",""))
    return (12*ft_) + in_
df['height'] = df['height'].apply(lambda x:parse_height(x))
# change 'party: cocktail' in 'rented for' to 'others'
df.loc[df['rented for'] == 'party: cocktail', df.columns == 'rented for'] = 'other'

--- map cups to numerical value ---
{'cups': {'a': 1, 'b': 2, 'c': 3, 'd': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8, 'j': 9}}


In [16]:
df=df.drop(columns=['user_id', 'item_id','review_date','review_summary','review_text','fit'])

In [17]:
df['category'].unique()

array(['romper', 'gown', 'dress', 'sheath', 'leggings', 'sweater',
       'jacket', 'shirtdress', 'jumpsuit', 'shift', 'top', 'shirt',
       'mini', 'skirt', 'maxi', 'pants', 'suit', 'coat', 'blouse',
       'trench', 'cape', 'bomber', 'blazer', 'vest', 'duster', 'ballgown',
       'tank', 'poncho', 'frock', 'tunic', 'cardigan', 'down', 'culottes',
       'midi', 'legging', 'print', 'pant', 'knit', 'culotte',
       'sweatshirt', 'peacoat', 'trouser', 'kaftan', 'overalls', 'jogger',
       'tee', 'combo', 'henley', 'blouson', 'pullover', 'turtleneck',
       'trousers', 'overcoat', 'hoodie', 't-shirt', 'caftan', 'tight',
       'kimono', 'cami', 'for', 'crewneck', 'skirts', 'parka',
       'buttondown', 'skort', 'sweatershirt', 'jeans', 'sweatpants'],
      dtype=object)

In [18]:
df.loc[df['category'].isin(['sweater','shirt','blouse','vest','tank','tunic','print','sweatshirt',
                            'tee','blouson','turtleneck','hoodie','t-shirt','cami','crewneck',
                            'buttondown','sweatershirt'
                           ]),df.columns=='category']='tops'
df.loc[df['category'].isin(['pants','down','culottes','pant','trouser','culotte','jogger',
                            'trousers','jeans', 'sweatpants'
                           ]),df.columns=='category']='bottoms'
df.loc[df['category'].isin(['ballgown']),df.columns=='category']='gown'
df.loc[df['category'].isin(['leggings','legging']),df.columns=='category']='active'
df.loc[df['category'].isin(['jacket','trench','cape','bomber','blazer','duster','poncho',
                            'cardigan','peacoat','pullover','overcoat','parka'
                           ]),df.columns=='category']='coat'
df.loc[df['category'].isin(['mini','midi','skirts']),df.columns=='category']='skirt'
df.loc[df['category'].isin(['jumpsuit']),df.columns=='category']='romper'
df.loc[df['category'].isin(['sheath','shift','maxi','shirtdress','frock','kaftan','caftan'
                           ]),df.columns=='category']='dress'
df.loc[df['category'].isin(['overalls','combo','henley','tight','kimono','for','skort']),df.columns=='category']='other'

In [19]:
df.info()
df= pd.get_dummies(df, columns=['body type','category','rented for'])


<class 'pandas.core.frame.DataFrame'>
Int64Index: 146381 entries, 0 to 192543
Data columns (total 10 columns):
age           146381 non-null float64
body type     146381 non-null object
bust size     146381 non-null int64
category      146381 non-null object
height        146381 non-null float64
rating        146381 non-null float64
rented for    146381 non-null object
size          146381 non-null int64
weight        146381 non-null int64
cups          146381 non-null int64
dtypes: float64(3), int64(4), object(3)
memory usage: 12.3+ MB


In [20]:
x=df.drop(columns=['rating'],axis=1)
y=df['rating']
x_train, x_test,y_train,y_test = train_test_split(x,y, test_size=0.33)


In [21]:
#Linear Regression
lg=LinearRegression(normalize=True).fit(x_train,y_train)
lg_train_pred=lg.predict(x_train)
lg_train_mse=mean_squared_error(y_train, lg_train_pred)
print('Linear Regression Train MSE:',lg_train_mse)
lg_test_pred=lg.predict(x_test)
lg_test_mse=mean_squared_error(y_test, lg_test_pred)
print('Linear Regression Test MSE:',lg_test_mse)

Linear Regression Train MSE: 2.0381677476356383
Linear Regression Test MSE: 2.0394645238896953


In [22]:
#Decision Tree Regressor
dt=DecisionTreeRegressor(min_samples_split=10,min_samples_leaf=10, max_depth=10).fit(x_train,y_train)
dt_train_pred=dt.predict(x_train)
dt_train_mse=mean_squared_error(y_train, dt_train_pred)
print('Decision Tree Regressor Train MSE:',dt_train_mse)
dt_test_pred=dt.predict(x_test)
dt_test_mse=mean_squared_error(y_test, dt_test_pred)
print('Decision Tree Regressor Test MSE:',dt_test_mse)

Decision Tree Regressor Train MSE: 1.988644428707625
Decision Tree Regressor Test MSE: 2.0671111922425105


In [23]:
#Random Forest regressor
rm=RandomForestRegressor(max_depth=10,n_estimators=100,min_samples_split=10,min_samples_leaf=10).fit(x_train,y_train)
rm_train_pred=rm.predict(x_train)
rm_train_mse=mean_squared_error(y_train, rm_train_pred)
print('Random Forest Regressor Train MSE:',rm_train_mse)
rm_test_pred=rm.predict(x_test)
rm_test_mse=mean_squared_error(y_test, rm_test_pred)
print('Random Forest Regressor Test MSE:',rm_test_mse)

Random Forest Regressor Train MSE: 1.9684521356665972
Random Forest Regressor Test MSE: 2.032091826358045


In [ ]:
#Gradient Boosting Regressor
gb=GradientBoostingRegressor(loss='ls', learning_rate=0.001, n_estimators=100, min_samples_split=10, max_depth=10).fit(x_train,y_train)
gb_train_pred=gb.predict(x_train)
gb_train_mse=mean_squared_error(y_train, gb_train_pred)
print('Gradient Boosting Regressor Train MSE:',gb_train_mse)
gb_test_pred=gb.predict(x_test)
gb_test_mse=mean_squared_error(y_test, gb_test_pred)
print('Gradient Boosting Regressor Test MSE:',gb_test_mse)